In [1]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.7 MB/s eta 0:00:00


In [2]:
import tiktoken
import tensorflow as tf
import numpy as np
import re
import heapq
import random
from tensorflow.keras  import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, LayerNormalization, Dense, Add, Concatenate, Dropout
from keras.saving import register_keras_serializable
from keras import config

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

token2vec = tiktoken.encoding_for_model("gpt-3.5-turbo")

Mounted at /content/drive


In [3]:
#global var
context_size = 100

In [4]:
class embedify(tf.keras.layers.Layer):
  # __NOTES__
  # The build() function is used to create weights that depend on the input shape, we M's dimensions set in constructor so it's all good
  # The get_config() function is used to return a dict of what input params are needed to load this model from a save
  def __init__(self, emb_dim, vocab_size, context_size, **kwargs):
    super().__init__(**kwargs)
    self.emb_dim = emb_dim
    self.vocab_size = vocab_size
    self.context_size = context_size
    self.M = self.add_weight(shape=(vocab_size, emb_dim), initializer='glorot_uniform', name='M', trainable=True)
    position = np.arange(context_size)[:, np.newaxis]
    div_term = np.exp(np.arange(0, emb_dim, 2) * -(np.log(10000.0) / emb_dim))
    pe = np.zeros((context_size, emb_dim))
    pe[:, 0::2] = np.sin(position * div_term)
    pe[:, 1::2] = np.cos(position * div_term)
    self.positional_encoding = tf.constant(pe[np.newaxis, :, :], dtype=tf.float32)

  def call(self, x):
    x = tf.cast(x, tf.int32)
    x = tf.one_hot(x, self.vocab_size)
    x = tf.matmul(x, self.M)
    seq_len = tf.minimum(self.context_size, tf.shape(x)[1])
    x = x[:, :seq_len, :] + self.positional_encoding[:, :seq_len, :]
    return x

  def get_config(self):
    config = super().get_config()
    config.update({"emb_dim": self.emb_dim, "vocab_size": self.vocab_size, "context_size":self.context_size})
    return config

  def compute_output_shape(self, input_shape):
    return input_shape + (self.emb_dim,)

class attentify(tf.keras.layers.Layer):
  # __NOTES__
  def __init__(self, emb_dim, head_dim, context_size, **kwargs):
    super().__init__(**kwargs)
    self.emb_dim = emb_dim
    self.head_dim = head_dim
    self.context_size = context_size
    self.Q = self.add_weight(shape=(emb_dim, head_dim), initializer='glorot_uniform', name='Q', trainable=True)
    self.K = self.add_weight(shape=(emb_dim, head_dim), initializer='glorot_uniform', name='K', trainable=True)

  def call(self, x):
    Qx = tf.matmul(x, self.Q)
    Kx = tf.matmul(x, self.K)
    A = tf.matmul(Qx, Kx, transpose_b=True) / tf.math.sqrt(tf.cast(self.head_dim, tf.float32))
    mask = tf.linalg.band_part(tf.ones_like(A), -1, 0)
    neg_inf = tf.fill(tf.shape(A), -1e-9)
    A = tf.where(mask == 1, A, neg_inf)
    A = tf.nn.softmax(A)
    x = tf.matmul(A, x) + x
    return x

  def get_config(self):
    config = super().get_config()
    config.update({"emb_dim": self.emb_dim, "head_dim": self.head_dim, "context_size":self.context_size})
    return config

  def compute_output_shape(self, input_shape):
    return input_shape

class MLPify(tf.keras.layers.Layer):
  # __NOTES__
  def __init__(self, emb_dim, expansion_multiplier, **kwargs):
    super().__init__(**kwargs)
    self.emb_dim = emb_dim
    self.expansion_multiplier = expansion_multiplier

  def call(self, x, training=False):
    x = self.denseUp(x)
    x = self.denseDown(x)
    x = self.dropout(x,training=training)
    return x

  def build(self, input_shape):
    self.denseUp = Dense(self.emb_dim*self.expansion_multiplier, activation="gelu")
    self.denseDown = Dense(self.emb_dim, activation="gelu")
    self.dropout = Dropout(0.1)

    self.denseUp.build(input_shape)  # input: (batch, context, emb_dim)
    up_out_shape = self.denseUp.compute_output_shape(input_shape)
    self.denseDown.build(up_out_shape)
    self.dropout.build(up_out_shape)
    super().build(input_shape)

  def compute_output_shape(self, input_shape):
    return input_shape

  def get_config(self):
    config = super().get_config()
    config.update({"emb_dim": self.emb_dim, "expansion_multiplier": self.expansion_multiplier})
    return config

  def get_config(self):
    config = super().get_config()
    config.update({"emb_dim": self.emb_dim, "expansion_multiplier": self.expansion_multiplier})
    return config

In [5]:
model = load_model(
    "/content/drive/MyDrive/shakespeareGPT.keras",
    custom_objects={"Custom>embedify": embedify,
                    "Custom>attentify": attentify,
                    "Custom>MLPify": MLPify})

In [6]:
def generate_next_word(text, gpt, top_word_range=2):
  input_tokens = token2vec.encode(text)[-context_size:]
  if len(input_tokens) < context_size:
      input_tokens = [0] * (context_size - len(input_tokens)) + input_tokens

  input_tensor = tf.convert_to_tensor([input_tokens], dtype=tf.int32)
  pdf = gpt.predict(input_tensor, verbose=0)
  heap = [(-pdf[0][i], i) for i in range(len(pdf[0]))]
  heapq.heapify(heap)
  pops = random.randint(0,top_word_range)
  for _ in range(pops):
    heapq.heappop(heap)
  next_token_id = heapq.heappop(heap)[1]
  next_word = token2vec.decode([next_token_id])
  return next_word

In [8]:
text = "Upon this fair " #prompt
for _ in range(150):
  text += generate_next_word(text, model)
print(text)

Upon this fair  I
 to my brother's the King. I have a woman'stis'd,
     And all the world of my lord,
     But I have the King of his Grace.
     I'll be not, I am not to me, I have not, I amazement
     And so far in my heart's a good.
  Lear. And I am glad of the matter?
  I'll have been a man, sir, sir.
    And let me to the world's a man.
     But let me, sir; I have a man, I'll make me, and I'll be a
  CLOWALO
     To make him to the King's a woman.
    I am not, and so
